In [2]:
import pandas as pd
import bs4,urllib
import requests
from tqdm import tqdm

In [3]:
data = pd.read_pickle('north_data.pickle')
user_profiles = []
for i in range(0,len(data)):
    q_links = data['Scraped_info'][i]['Profile_Link']
    for link in q_links:
        if link not in user_profiles:
            user_profiles.append(link)

In [ ]:
profiles_df = pd.DataFrame(columns=['Profile_Link','Question','Date','Answer'])#'Upvotes'])
for i in tqdm(range(0,len(user_profiles))):
    page = urllib.request.urlopen(user_profiles[i]).read()
    soup = bs4.BeautifulSoup(page,'lxml')
    all_questions = soup.findAll('div',{'class':'pagedlist_item'})
    for i in range(0,len(all_questions)):
        try:
            question = all_questions[i].find('a',{'class':'question_link'}).get_text()
            date = all_questions[i].find('a',{'class':'answer_permalink'}).get_text()
            answer_more_link = 'https://www.quora.com'+all_questions[i].find('a',{'class':'ui_qtext_more_link'})['href']
            answer_page = urllib.request.urlopen(answer_more_link)
            answer_soup = bs4.BeautifulSoup(answer_page,'lxml')
            full_answer = answer_soup.find('div',{'class':'ui_qtext_expanded'}).get_text()
            #upvotes = answer_soup.findAll('span',{'class':'ui_button_count_optimistic_counts'}).get_text()
        except:
            pass
        curr_row = {'Profile_Link':i ,
                    'Question':question,
                    'Date':date,
                    'Answer':full_answer }
        profiles_df = profiles_df.append(curr_row,ignore_index=True)

 14%|██████████▋                                                                    | 69/510 [38:34<4:20:32, 35.45s/it]